## Start point
This analysis gives me the right columns that I need to work with to increase the chances of my results. 
The columns are: survived, sex, had_cabin, is_adult, p_class_edit, strong_title, sibsp_presence, and parch_presence, 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_columns', None)  # Show all columns (if you need this)

In [2]:
# Importing data sets 
dataset = pd.read_csv('train.csv')

In [3]:
#Get Titles and strong_titles columns
titles = []
name = dataset['Name']
for na in name:
    name_lst = na.split(',')
    name_lst_2 = name_lst[1].split('.')
    name_lst_3 = name_lst_2[0]
    title = name_lst_3.replace("the", "").strip()  
    titles.append(title)
dataset["Titles"] = titles
normal_titles = ["Mr", "Mrs", "Miss", "Master"]
dataset["strong_title"] = np.where(
    dataset['Titles'].isin(normal_titles), 0, 1
)


In [4]:
#adding Had_Cabin column
cabin_presence = []
cabin = dataset['Cabin']
for cab in cabin:
    if pd.isna(cab):
        presence = 0
    else:
        presence = 1
    cabin_presence.append(presence)

dataset["Had_Cabin"] = cabin_presence


In [5]:
#Adding the is_adult column
dataset['Age'] = dataset['Age'].fillna(-1)

dataset['is_adult'] = np.where(
    (dataset['Age'] < 13), 0,
    np.where(
        (dataset['Age'] > 18), 1,
        np.where(
            (dataset['Titles'] == 'Master'), 0,
            1
        )
    )
)


In [6]:
#adding p_class_edit column
#Add Passenger class column in categorical format
pass_class = []
p_class = dataset['Pclass']
for p in p_class:
    if p == 1:
        class_ = 'First'
    elif p==2:
        class_ = 'Second'
    else:
        class_ = 'Third'
    pass_class.append(class_)
dataset['P_Class_Edit'] = pass_class

In [7]:
#adding the sibsp_presence column
sibsp_presence = []
sibsp = dataset['SibSp']
for ss in sibsp:
    if ss > 0:
        presence = 1
    elif ss == 0:
        presence = 0
    else:
        presence = "null"
    sibsp_presence.append(presence)

dataset["sibsp_presence"] = sibsp_presence

In [8]:
#whether they had a parent or child
parch_presence = []
parch = dataset['Parch']
for pch in parch:
    if pch > 0:
        presence = 1
    elif pch == 0:
        presence = 0
    else:
        presence = "null"
    parch_presence.append(presence)

dataset["parch_presence"] = parch_presence

In [9]:
# Filter rows where Titles == 'Master' and select the Age column
master_ages = dataset[dataset['Titles'] == 'Master']['Age']

# Calculate the average of the Age column
average_age = master_ages.mean()
max_age = master_ages.max()
min_age = master_ages.min()

print("The average age is:", average_age)
print("The max age is:", max_age)
print("The mix age is:", min_age)



The average age is: 4.01675
The max age is: 12.0
The mix age is: -1.0


In [10]:
ages_titles = dataset[["Titles", "Age"]]
mean_ages = ages_titles.groupby('Titles')['Age'].mean()

print(mean_ages)

Titles
Capt        70.000000
Col         58.000000
Countess    33.000000
Don         40.000000
Dr          35.857143
Jonkheer    38.000000
Lady        48.000000
Major       48.500000
Master       4.016750
Miss        17.269231
Mlle        24.000000
Mme         24.000000
Mr          24.687621
Mrs         30.880000
Ms          28.000000
Rev         43.166667
Sir         49.000000
Name: Age, dtype: float64


## Analysis
The fact is that in the times of the titanic, a child was not under 18. A person was considered an adult when they were above 12. I will put 13 just to be safe. 

In [11]:
#Get the useful data
useful_data = dataset[['Survived', 'Sex', 'strong_title', 'Had_Cabin', 'is_adult', 'P_Class_Edit', 'sibsp_presence', 'parch_presence']]
useful_data

,Survived,Sex,strong_title,Had_Cabin,is_adult,P_Class_Edit,sibsp_presence,parch_presence
0,0,male,0,0,1,Third,1,0
1,1,female,0,1,1,First,1,0
2,1,female,0,0,1,Third,0,0
3,1,female,0,1,1,First,1,0
4,0,male,0,0,1,Third,0,0
...,...,...,...,...,...,...,...,...
886,0,male,1,0,1,Second,0,0
887,1,female,0,1,1,First,0,0
888,0,female,0,0,0,Third,1,1
889,1,male,0,1,1,First,0,0


In [ ]:
useful_data.info()

## Analysis
There are no missing values here. 
However, I have a few object datatypes which can cause a problem when it comes to manipulating the data. 
I can start by getting the dependent and independent variabe

In [12]:
# Splitting features (X) and target variable (y)
X = useful_data.iloc[:, 1:]  # Select all columns except the first one (Survived)
y = useful_data.iloc[:, 0]  # Select the first column (Survived)
X.head()

,Sex,strong_title,Had_Cabin,is_adult,P_Class_Edit,sibsp_presence,parch_presence
0,male,0,0,1,Third,1,0
1,female,0,1,1,First,1,0
2,female,0,0,1,Third,0,0
3,female,0,1,1,First,1,0
4,male,0,0,1,Third,0,0


In [14]:
# Dealing with Categorical Data
# X = pd.get_dummies(X, columns=['Sex'], drop_first=True)
# X = pd.get_dummies(X, columns=['P_Class_Edit'], drop_first=True)
X

,strong_title,Had_Cabin,is_adult,sibsp_presence,parch_presence,Sex_male,P_Class_Edit_Second,P_Class_Edit_Third
0,0,0,1,1,0,True,False,True
1,0,1,1,1,0,False,False,False
2,0,0,1,0,0,False,False,True
3,0,1,1,1,0,False,False,False
4,0,0,1,0,0,True,False,True
...,...,...,...,...,...,...,...,...
886,1,0,1,0,0,True,True,False
887,0,1,1,0,0,False,False,False
888,0,0,0,1,1,False,False,True
889,0,1,1,0,0,True,False,False


In [15]:
# Splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [16]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [20]:
##The ANN Model
#Part 2 - Building the ANN
import keras
from keras.models import Sequential
from keras.layers import Dense 

#create the classifier
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=8))

# Adding the second hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

# Adding the output layer
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Part 3 - Training the ANN

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [25]:
# # Training the ANN on the Training set
classifier.fit(X_train, y_train, batch_size = 32, epochs = 350)                   



Epoch 1/350
21/21 [==============================] - 0s 6ms/step - loss: 0.4000 - accuracy: 0.8323
Epoch 2/350
21/21 [==============================] - 0s 8ms/step - loss: 0.4003 - accuracy: 0.8338
Epoch 3/350
21/21 [==============================] - 0s 5ms/step - loss: 0.4004 - accuracy: 0.8338
Epoch 4/350
21/21 [==============================] - 0s 5ms/step - loss: 0.4001 - accuracy: 0.8338
Epoch 5/350
21/21 [==============================] - 0s 6ms/step - loss: 0.3998 - accuracy: 0.8338
Epoch 6/350
21/21 [==============================] - 0s 7ms/step - loss: 0.3999 - accuracy: 0.8338
Epoch 7/350
21/21 [==============================] - 0s 6ms/step - loss: 0.4002 - accuracy: 0.8323
Epoch 8/350
21/21 [==============================] - 0s 4ms/step - loss: 0.3994 - accuracy: 0.8338
Epoch 9/350
21/21 [==============================] - 0s 8ms/step - loss: 0.3997 - accuracy: 0.8338
Epoch 10/350
21/21 [==============================] - 0s 9ms/step - loss: 0.4003 - accuracy: 0.8323
Epoch 11/

In [28]:
##Predicting the Results 
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)


7/7 [==============================] - 0s 7ms/step


In [ ]:
#Fitting kernel SVM to the training set 
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state=0, C = 1, gamma = 0.5, degree = 2)
classifier.fit(X_train, y_train)

In [ ]:
#Implementing Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [
    {"C":[1,10,100,1000], 'kernel':['linear']},
    {"C":[1,10,100,1000], 'kernel':['rbf'], 'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]}
    ]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [ ]:
# ## Applying k-fold cross validation 
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv = 10, n_jobs=-1)
print("accuracies mean", accuracies.mean())
print("accuracies std", accuracies.std())


In [24]:
##Predicting the Results 
y_pred = classifier.predict(X_test)
print(y_pred)

7/7 [==============================] - 0s 6ms/step
[[0.09308018]
 [0.09308018]
 [0.18683742]
 [0.98464745]
 [0.48560986]
 [0.07466937]
 [0.9869864 ]
 [0.96754634]
 [0.44651553]
 [0.78192896]
 [0.0905755 ]
 [0.87725705]
 [0.09308018]
 [0.9808169 ]
 [0.9869864 ]
 [0.56897664]
 [0.1526569 ]
 [0.10969781]
 [0.1526569 ]
 [0.19943239]
 [0.06133896]
 [0.98633957]
 [0.09308018]
 [0.14405926]
 [0.56897664]
 [0.92870253]
 [0.1526569 ]
 [0.56897664]
 [0.92744756]
 [0.33747798]
 [0.1526569 ]
 [0.87725705]
 [0.1526569 ]
 [0.07466937]
 [0.0905755 ]
 [0.5082439 ]
 [0.1526569 ]
 [0.06133896]
 [0.06133896]
 [0.09398162]
 [0.31009883]
 [0.1526569 ]
 [0.09308018]
 [0.07166749]
 [0.9832993 ]
 [0.09308018]
 [0.09308018]
 [0.92870253]
 [0.06133896]
 [0.31009883]
 [0.07466937]
 [0.25288135]
 [0.98548573]
 [0.1526569 ]
 [0.07466937]
 [0.06340782]
 [0.3864025 ]
 [0.33747798]
 [0.18683742]
 [0.10575612]
 [0.1526569 ]
 [0.48560986]
 [0.9832993 ]
 [0.1590882 ]
 [0.48560986]
 [0.1526569 ]
 [0.97597516]
 [0.3718482

In [29]:
## Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[125  14]
 [ 33  51]]


## Handling unknown data

The model is now fully developed and now I want to try and use it on the test data. 

In [30]:
#uplead the data
test_dataset = pd.read_csv('test.csv')
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [31]:
#Preprocess data

#Titles
test_titles = []
test_name = test_dataset['Name']
for na in test_name:
    name_lst = na.split(',')
    name_lst_2 = name_lst[1].split('.')
    name_lst_3 = name_lst_2[0]
    title = name_lst_3.replace("the", "").strip() 
    test_titles.append(title)
    
test_dataset["Titles"] = test_titles
normal_titles = ["Mr", "Mrs", "Miss", "Master"]
test_dataset["strong_title"] = np.where(
    test_dataset['Titles'].isin(normal_titles), 0, 1
)

# Cabin
test_cabin_presence = []
test_cabin = test_dataset['Cabin']
for cab in test_cabin:
    if pd.isna(cab):
        presence = 0
    else:
        presence = 1
    test_cabin_presence.append(presence)

test_dataset["Had_Cabin"] = test_cabin_presence

#is_adult
test_dataset['Age'] = test_dataset['Age'].fillna(-1)

test_dataset['is_adult'] = np.where(
    (test_dataset['Age'] < 18), 0,
    np.where(
        (test_dataset['Age'] > 18), 1,
        np.where(
            (test_dataset['Titles'] == 'Master'), 0,
            1
        )
    )
)

#P_Class
test_pass_class = []
test_p_class = test_dataset['Pclass']

for p in test_p_class:
    if p == 1:
        class_ = 'First'
    elif p==2:
        class_ = 'Second'
    else:
        class_ = 'Third'    
    test_pass_class.append(class_)
test_dataset['P_Class_Edit'] = test_pass_class

#sibsp_presence
test_sibsp_presence = []
sibsp = test_dataset['SibSp']
for ss in sibsp:
    if ss > 0:
        presence = 1
    elif ss == 0:
        presence = 0
    else:
        presence = "null"
    test_sibsp_presence.append(presence)
test_dataset["sibsp_presence"] = test_sibsp_presence

#Parch
test_parch_presence = []
test_parch = test_dataset['Parch']
for pch in test_parch:
    if pch > 0:
        presence = 1
    elif pch == 0:
        presence = 0
    else:
        presence = "null"
    test_parch_presence.append(presence)

test_dataset["parch_presence"] = test_parch_presence





In [32]:
##Get useful data
test_useful_data = test_dataset[['Sex', 'strong_title', 'Had_Cabin', 'is_adult', 'P_Class_Edit', 'sibsp_presence', 'parch_presence']]


In [34]:
# test_useful_data = pd.get_dummies(test_useful_data, columns=['Sex'], drop_first=True)
# test_useful_data = pd.get_dummies(test_useful_data, columns=['P_Class_Edit'], drop_first=True)
test_useful_data

,strong_title,Had_Cabin,is_adult,sibsp_presence,parch_presence,Sex_male,P_Class_Edit_Second,P_Class_Edit_Third
0,0,0,1,0,0,True,False,True
1,0,0,1,1,0,False,False,True
2,0,0,1,0,0,True,True,False
3,0,0,1,0,0,True,False,True
4,0,0,1,1,1,False,False,True
...,...,...,...,...,...,...,...,...
413,0,0,0,0,0,True,False,True
414,1,1,1,0,0,False,False,False
415,0,0,1,0,0,True,False,True
416,0,0,0,0,0,True,False,True


In [49]:
#feature scaling and prediction
test_useful_data = sc.transform(test_useful_data)
test_pred = classifier.predict(test_useful_data)

##Predicting the Results 
test_pred = (test_pred > 0.5).astype(int)
test_pred


14/14 [==============================] - 0s 9ms/step


/home/freddie/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [40]:
# Convert test_pred to a DataFrame with the column name 'Survived'
test_pred_df = pd.DataFrame({'Survived': test_pred})
test_pred_df

ValueError: Per-column arrays must each be 1-dimensional

In [ ]:
# Combine test_id and test_pred DataFrames
test_id = test_dataset['PassengerId']
result = pd.concat([test_id, test_pred_df], axis=1)
result

In [ ]:
# Save the combined DataFrame to a CSV file
result.to_csv('v3_kernalSVM.csv', index=False)